In [1]:
import os
import os.path as osp

import random

from tqdm import tqdm

import numpy as np

import open3d as o3d

import utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd_path = "data/kinth/pointcloud/1720509720.141261339.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = pcd.voxel_down_sample(voxel_size=0.01)
pcd = pcd.remove_radius_outlier(nb_points=25, radius=0.2)[0] # (pcd, new indexed from old)


points = np.asarray(pcd.points)
points = points - points.mean(axis=0)
print("max, min:", points.max(axis=0), points.min(axis=0))
print(f"before voxelized: {len(points)}")
points, _, _, _ = utils.voxel_downsample(points, 0.02, use_avg=False)
print("max, min:", points.max(axis=0), points.min(axis=0))
print(f"after voxelized: {len(points)}")

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
max, min: [4.71142875 3.22041954 1.96897121] [-3.35557116 -4.00158058 -1.04102878]
before voxelized: 12234
max, min: [8.07 7.23 3.01] [0.01 0.01 0.01]
after voxelized: 7677


In [24]:
radius = 0.15

colors = np.ones((len(points), 3), dtype=np.int32) * 75
search_tree = o3d.geometry.KDTreeFlann(utils.npy2o3d(points))
query_idx_list = (np.random.rand(3) * len(points)).astype(np.int32)
for query_idx in tqdm(range(len(points)), total=len(points), ncols=100):
    query = points[query_idx]
    neighbour_num, neighbour_idx_list, _ = search_tree.search_radius_vector_3d(query, radius)
    if neighbour_num <= 3:
        continue
    eigvals, eigvecs = utils.pca_k(points[neighbour_idx_list], 3)
    assert eigvals[0] >= eigvals[1] and eigvals[1] >= eigvals[2]
    eigval_ratio = np.array([
        (eigvals[0] - eigvals[1]) / (eigvals[0] + 1e-9),
        (eigvals[1] - eigvals[2]) / (eigvals[1] + 1e-9),
        1e-9
    ])
    
    colors[query_idx] = (eigval_ratio / eigval_ratio.sum() * 255.0).astype(np.int32)

utils.npy2ply(points, colors, "data/output/test.ply")

100%|████████████████████████████████████████████████████████| 7677/7677 [00:00<00:00, 11847.37it/s]


In [4]:
def search_radius_bf(points: np.ndarray, targets: np.ndarray, inner_radius:float, outer_radius: float):
    '''
    find neighbours with distance in between the range [exclude, include].
    
    params:
    -
    * points (np.ndarray) - all points in the scene
    * targets (np.ndarray) - all points to be queried the neighbours
    * inner_radius (float) - lower bound of the neighbour range
    * outer_radius (float) - upper bound of the neighbour range
    
    return:
    * list(np.ndarray) - list of neighbours of each target
    '''
    assert inner_radius is not None and outer_radius is not None
    
    assert points.shape[-1] == targets.shape[-1]
    if len(targets.shape) == 1:
        targets = np.expand_dims(targets, axis=0)
        
    result = []
    for target in targets:
        dist = np.linalg.norm(points - target, axis=1)
        mask = (dist >= inner_radius) & (dist <= outer_radius)
        result.append(points[mask])
    

In [7]:
def sin_batch(bat1: np.ndarray, bat2: np.ndarray):
    cos_batch_result = utils.cos_batch(bat1, bat2)
    assert not np.all(np.isnan(cos_batch_result))
    return np.sqrt(1 - cos_batch_result**2)


wup = np.array([0, 0, 1])
border = 0.5
colors = np.ones((len(points), 3), dtype=np.int32) * 50
queries = (np.random.rand(3) * len(points)).astype(np.int32)
for query_idx in tqdm(range(len(points)), total=len(points), ncols=100):
    query = points[query_idx]
    # mask = (np.abs(points - query) < border / 2.0).astype(np.int32).sum(axis=1) == 3
    mask = (
        (np.abs(points[:, 0] - query[0]) < border / 2.0) &
        (np.abs(points[:, 1] - query[1]) < border / 2.0) &
        (np.abs(points[:, 2] - query[2]) < border)
    )
    mask[query_idx] = False
    vicinity = points[mask]
    if len(vicinity) < 3:
        continue
    feat = (1.0 - sin_batch(points[mask] - query, wup)).mean()
    colors[query_idx] = int(feat * 255.0)

utils.npy2ply(points, colors, "test.ply")

100%|████████████████████████████████████████████████████████| 7677/7677 [00:00<00:00, 10986.61it/s]


In [5]:
import numpy as np

a = np.array([0, 1, 0])
b = np.array([0, 0, 1])

c = a / b
c = np.nan_to_num(c, nan=0, posinf=1, neginf=-1)

print(c)

[0. 1. 0.]


/tmp/ipykernel_3213199/1020614606.py:6: RuntimeWarning: divide by zero encountered in divide
  c = a / b
/tmp/ipykernel_3213199/1020614606.py:6: RuntimeWarning: invalid value encountered in divide
  c = a / b
